Данная задача была выбрана для визуализации потому, что мало где уделяется время табличной визуализации.
Например, как в данном случае, табличные данные из себя представляют результаты AB теста, где больше одной экспериментальной группы, где рисовать графики по каждой метрики для всех групп и может не дать сконцентрироваться на самом важном.

Поэтому очень удобно визуализировать таблицу  и делать акцент цветами на важных данных.

In [13]:
import pandas as pd
import pyodbc
from ipywidgets import IntProgress
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from math import ceil
from statsmodels.stats.proportion import proportions_ztest, proportions_chisquare, test_proportions_2indep

In [2]:
df_cntrl = pd.read_csv('A_res_test.csv')
df_exprmnt_ = pd.read_csv('B_res_test.csv')

In [3]:
display(df_cntrl)

,segment_N,group_,ExpConversionPercnet,Customers,Buyers,Amount
0,2,CG1,2.398667,75000,1799,6.081564e+06
1,5,CG1,0.814667,75000,611,2.094266e+06
2,1,CG2,7.224000,75000,5418,1.875245e+07
3,4,CG2,2.436000,75000,1827,5.369454e+06
4,3,CG1,2.922667,75000,2192,6.478885e+06
5,3,CG2,3.137333,75000,2353,6.596179e+06
6,1,CG1,7.185333,75000,5389,1.955974e+07
7,4,CG1,2.412000,75000,1809,4.873048e+06
8,2,CG2,2.214667,75000,1661,4.964514e+06
9,5,CG2,0.782667,75000,587,1.757914e+06


In [4]:
df_exprmnt = df_exprmnt_.sort_values(['segment_N', 'group_', 'CHANNEL', 'STATUS_']).query('STATUS_ != 0')

In [14]:
df_exprmnt_.groupby(['segment_N', 'group_',]).agg({'Customers':'sum', 'Buyers':'sum', 'Marg':'sum'})

Customers  Buyers          Marg
segment_N group_                                 
1         ExpG1       74999    5862  7.350771e+06
          ExpG2       75000    6414  9.134277e+06
          ExpG3       75000    5478  3.061805e+06
2         ExpG1       75000    1760  2.303603e+06
          ExpG2       75000    1827  2.398518e+06
          ExpG3       75000    1507  8.515943e+05
3         ExpG1       75000    2499  2.592659e+06
          ExpG2       75000    2408  2.766411e+06
          ExpG3       75000    2412  9.403937e+05
4         ExpG1       75000    1794  1.824358e+06
          ExpG2       75000    1705  1.790713e+06
          ExpG3       75000    1685  8.731102e+05
5         ExpG1       75000     618  6.836939e+05
          ExpG2       75000     600  5.800301e+05
          ExpG3       75000     680  3.063963e+05

# Группировка по сегмантам, группам.
Сравнение с контрольной группой 2

In [15]:
def GetTrustDifference(contrl_conv, expr_conv,
                       n_cntrl_customers, n_expr_customers,
                       n_cntrl_buyers, n_expr_buyers): #return Bool, n_customer, delta what we have
    required_n = 1e6
    h = 0.0001
    beta = 0.05
    alpha = 0.05
    a = contrl_conv 
    b = expr_conv 
    successes = [n_cntrl_buyers, n_expr_buyers]
    nobs = [n_cntrl_customers, n_expr_customers]
    #pval = proportions_chisquare(successes, nobs=nobs)[1]
    #pval = proportions_ztest(successes, nobs=nobs)[1]
    pval = test_proportions_2indep(successes[0], nobs[0],successes[1], nobs[1])[1]
    
    if(pval > 0.05):
        return [False, 0, round((b - a)*100, 6), round(pval, 6)]
    else:
        return [True, 0, round((b - a)*100, 6), round(pval, 6)]
    
    


In [17]:
df_expr_grp_seg_gr =  df_exprmnt.groupby(['segment_N', 'group_']).agg({'Customers':'sum',
                                 'Buyers':'sum'})
df_expr_grp_seg_gr

Customers  Buyers
segment_N group_                   
1         ExpG1       65315    5258
          ExpG2       65739    5792
          ExpG3       28502    2276
2         ExpG1       66467    1650
          ExpG2       66831    1699
          ExpG3       23852     550
3         ExpG1       61279    2189
          ExpG2       66321    2228
          ExpG3       22637     811
4         ExpG1       64333    1640
          ExpG2       64833    1553
          ExpG3       24596     676
5         ExpG1       57957     537
          ExpG2       57249     524
          ExpG3       18289     209

In [18]:
df_cntrl_grp_seg_gr = df_cntrl.groupby(['segment_N', 'group_']).agg({'Customers':'sum',
                                 'Buyers':'sum'})
df_cntrl_grp_seg_gr

Customers  Buyers
segment_N group_                   
1         CG1         75000    5389
          CG2         75000    5418
2         CG1         75000    1799
          CG2         75000    1661
3         CG1         75000    2192
          CG2         75000    2353
4         CG1         75000    1809
          CG2         75000    1827
5         CG1         75000     611
          CG2         75000     587

In [19]:
n = max(df_cntrl.segment_N.values)
n

5

In [20]:
expr_grps = df_expr_grp_seg_gr.loc[1].T.columns.tolist()
cntrl_grps = df_cntrl_grp_seg_gr.loc[1].T.columns.tolist()
i = 0
res_arr = []
for c_grp in cntrl_grps:
    cntrl = df_cntrl.query(f'group_ == "{c_grp}"').groupby('segment_N').agg({'Customers':'sum',
                                                                             'Buyers':'sum',
                                                                             'Amount':'sum'})
    cntrl['conv'] = cntrl['Buyers'] / cntrl['Customers']
    cntrl['Mean'] = cntrl['Amount'] / cntrl['Buyers']
    for exp_grp in expr_grps:
        exprmnt = df_exprmnt.query(f'group_ == "{exp_grp}"').groupby('segment_N').agg({'Customers':'sum',
                                                                                       'Buyers':'sum',
                                                                                       'Marg':'sum'})
        exprmnt['Cost_Mess'] = df_exprmnt_.query(f'group_ == "{exp_grp}"').groupby('segment_N').agg({'Cost_Mess':'sum'})
        exprmnt['Cost_Mess_S'] = df_exprmnt_.query(f'group_ == "{exp_grp}"').groupby('segment_N').agg({'Cost_Mess_S':'sum'})
        exprmnt['conv'] = exprmnt['Buyers'] / exprmnt['Customers']
        for s in range(0, n):
            res = GetTrustDifference(cntrl.loc[s+1]['conv'], exprmnt.loc[s+1]['conv'],
                                     cntrl.loc[s+1]['Customers'], exprmnt.loc[s+1]['Customers'], 
                                     cntrl.loc[s+1]['Buyers'], exprmnt.loc[s+1]['Buyers'])

            res_arr.append([s + 1, #0
                            c_grp, #1
                            exp_grp, #2
                            res[0], #3
                            res[1], #4
                            round(res[2], 6), #5
                            cntrl.loc[s+1]['conv'], #6
                            exprmnt.loc[s+1]['conv'], #7
                            res[3], #8
                            round(res[2], 6) / 100 * exprmnt.loc[s+1]['Customers'] * cntrl.loc[s+1]['Mean'], #9
                            round(res[2], 6) / 100  * exprmnt.loc[s+1]['Customers'] * cntrl.loc[s+1]['Mean'] * 0.45, #10
                            round(res[2], 6) / 100 * exprmnt.loc[s+1]['Customers'] * cntrl.loc[s+1]['Mean'] * 0.45 - \
                                exprmnt.loc[s+1]['Cost_Mess'],                                                #11
                            (round(res[2], 6) / 100 * exprmnt.loc[s+1]['Customers'] * cntrl.loc[s+1]['Mean'] * 0.45 - \
                                exprmnt.loc[s+1]['Cost_Mess']) / exprmnt.loc[s+1]['Customers'],               #12
                            (round(res[2], 6)/ 100  * exprmnt.loc[s+1]['Customers'] * cntrl.loc[s+1]['Mean'] * 0.45 - \
                                exprmnt.loc[s+1]['Cost_Mess']) /  exprmnt.loc[s+1]['Cost_Mess'],               #13
                            exprmnt.loc[s+1]['Cost_Mess'],                                                   #14
                            (round(res[2], 6)/ 100  * exprmnt.loc[s+1]['Customers'] * cntrl.loc[s+1]['Mean'] * 0.45 - \
                                exprmnt.loc[s+1]['Cost_Mess_S']) /  exprmnt.loc[s+1]['Cost_Mess_S'], #15
                            exprmnt.loc[s+1]['Cost_Mess_S']])


res_arr = np.array(res_arr)

In [22]:
df1_res = pd.DataFrame()
df1_res['Seg'] = res_arr.T[0]
df1_res['CntrlGrp'] = res_arr.T[1]
df1_res['ExpGrp'] = res_arr.T[2]
df1_res['Flag'] = res_arr.T[3]
#df1_res['requared'] = res_arr.T[4]
df1_res['delta%'] = np.array(list(map(float,res_arr.T[5])))
df1_res['CntrlConv%'] = np.array(list(map(float,res_arr.T[6]))) * 100
df1_res['ExprConv%'] = np.array(list(map(float,res_arr.T[7]))) * 100
df1_res['p_value'] = np.round(list(map(float,res_arr.T[8])), 6)
df1_res['Относительное_изменение_конверсии%'] = df1_res['delta%']  / df1_res['CntrlConv%'] * 100
df1_res['Затраты на коммуникации'] = np.array(list(map(float,res_arr.T[14])))
df1_res['Инкремент'] = np.array(list(map(float,res_arr.T[9])))
df1_res['Маржинальность инкремента'] = np.array(list(map(float,res_arr.T[10])))
df1_res['Маржинальность с вычетом затрат коммуникаций (Истинная доходность)'] = np.array(list(map(float,res_arr.T[11])))
df1_res['Истинная доходность с одного человека'] = np.array(list(map(float,res_arr.T[12])))
df1_res['ROI%'] = np.array(list(map(float,res_arr.T[13]))) * 100
df1_res['Затраты на коммуникации *'] = np.array(list(map(float,res_arr.T[16])))
df1_res['ROI% *'] = np.array(list(map(float,res_arr.T[15])))

In [23]:
df1_res.query('(CntrlGrp == "CG2")').sort_values('Seg').rename(columns = {'Seg':"Сегмент",
                                        'CntrlGrp' : 'Контроль',
                                        'ExpGrp' : 'Эксперимент',
                                        'delta%' : 'Разность конверсий %',
                                        'CntrlConv%' : 'Конверсия контроля %',
                                        'ExprConv%' : 'Конверсия эксперимента %',
                                        'Относительное_изменение_конверсии%' : 
                                        'Относительное изменение конверсии %'}).style.hide_index().bar(align = 'zero'
                                                       , subset = ['Относительное изменение конверсии %'
                                                                  ,'Маржинальность с вычетом затрат коммуникаций (Истинная доходность)'
                                                                  ,'ROI%']
                                                       , color=['red','#FFA07A']
                                                       ).background_gradient(axis=0, 
                                                          gmap=df1_res['Flag'].map({'False':0,
                                                                                    'True' : 1}), 
                                                        cmap = 'Pastel1').format(precision=3).hide_columns(['Flag',
                                                                                                               'p_value', 
                                                                                                               'Затраты на коммуникации', 
                                                                                                               'Инкремент',
                                                                                                               'Маржинальность инкремента',
                                                                                                               'Истинная доходность с одного человека',
                                                                                                               'Затраты на коммуникации *',
                                                                                                               'ROI% *'])



Сегмент,Контроль,Эксперимент,Разность конверсий %,Конверсия контроля %,Конверсия эксперимента %,Относительное изменение конверсии %,Маржинальность с вычетом затрат коммуникаций (Истинная доходность),ROI%
1,CG2,ExpG1,0.826,7.224,8.050,11.437,725735.868,632.357
1,CG2,ExpG2,1.587,7.224,8.811,21.963,1489507.732,1103.339
1,CG2,ExpG3,0.761,7.224,7.985,10.540,303802.171,888.248
2,CG2,ExpG1,0.268,2.215,2.482,12.091,122497.472,104.806
2,CG2,ExpG2,0.328,2.215,2.542,14.791,159440.615,118.104
2,CG2,ExpG3,0.091,2.215,2.306,4.119,641.654,2.242
3,CG2,ExpG1,0.435,3.137,3.572,13.861,219470.548,188.092
3,CG2,ExpG2,0.222,3.137,3.359,7.079,50803.262,37.632
3,CG2,ExpG3,0.445,3.137,3.583,14.193,99995.941,368.114
4,CG2,ExpG1,0.113,2.436,2.549,4.648,-19731.302,-16.999


Посчитав все, видим следующую картину:
     
     - Персиковым цветом выделены статистически не значимые результаты, при рассмотрении результатов их нельзя удалить из сводной таблицы, но таким образом обратим внимание на отсутствие статистической значимости;
     - Благодаря визуализации столбцом с помощью полосы уровня визуально становится понятно, где отрицательная выгода, где положительная и каков ее уровень;